In [1]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

In [2]:
import os

In [3]:
import tensorflow as tf
import numpy as np
import scipy
import time
from tensorflow.python.client import device_lib

In [4]:
device_lib.list_local_devices()

[name: "/cpu:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 14367593428700548487, name: "/gpu:0"
 device_type: "GPU"
 memory_limit: 7404299879
 locality {
   bus_id: 1
 }
 incarnation: 5155361092450031641
 physical_device_desc: "device: 0, name: GeForce GTX 1080, pci bus id: 0000:03:00.0"]

In [5]:
from model import Model_S2VT
from data_generator import Data_Generator
from inference_util import Inference

In [6]:
import inception_base

In [7]:
import configuration

In [8]:
data_config = configuration.DataConfig().config

In [9]:
data_gen = Data_Generator(data_config["processed_video_dir"],
                          data_config["caption_file"],
                          data_config["unique_frequency_cutoff"],
                          data_config["max_caption_length"])

In [17]:
data_gen.load_vocabulary(data_config["caption_data_dir"])
data_gen.load_dataset(data_config["caption_data_dir"])

['a',
 'cartoon',
 'animals',
 'runs',
 'through',
 'an',
 'ice',
 'cave',
 'in',
 'a',
 'video',
 'game',
 'a',
 'cartoon',
 'character',
 'runs',
 'around',
 'inside',
 'of',
 'a',
 'video',
 'game',
 'a',
 'character',
 'is',
 'running',
 'in',
 'the',
 'snow',
 'a',
 'person',
 'plays',
 'a',
 'video',
 'game',
 'centered',
 'around',
 'ice',
 'age',
 'the',
 'movie',
 'a',
 'person',
 'plays',
 'online',
 'and',
 'records',
 'themselves',
 'a',
 'scene',
 'from',
 'the',
 'ice',
 'age',
 'video',
 'game',
 'is',
 'shown',
 'a',
 'video',
 'game',
 'character',
 'is',
 'jumping',
 'about',
 'in',
 'a',
 'cave',
 'a',
 'video',
 'game',
 'of',
 'a',
 'little',
 'animal',
 'running',
 'through',
 'an',
 'ice',
 'tunnel',
 'a',
 'video',
 'game',
 'of',
 'a',
 'small',
 'animal',
 'a',
 'video',
 'shows',
 'gameplay',
 'from',
 'ice',
 'age',
 'a',
 'weasel',
 'runs',
 'around',
 'in',
 'an',
 'icy',
 'cave',
 'an',
 'animal',
 'runs',
 'through',
 'an',
 'icy',
 'tunnel',
 'in',
 'a'

In [ ]:
data_gen.build_basic_dataset()

In [ ]:
model_config = configuration.ModelConfig(data_gen).config

In [ ]:
model = Model_S2VT( num_frames = model_config["num_frames"],
                    image_width = model_config["image_width"],
                    image_height = model_config["image_height"],
                    image_channels = model_config["image_channels"],
                    num_caption_unroll = model_config["num_caption_unroll"],
                    num_last_layer_units = model_config["num_last_layer_units"],
                    image_embedding_size = model_config["image_embedding_size"],
                    word_embedding_size = model_config["word_embedding_size"],
                    hidden_size_lstm1 = model_config["hidden_size_lstm1"],
                    hidden_size_lstm2 = model_config["hidden_size_lstm2"],
                    vocab_size = model_config["vocab_size"],
                    initializer_scale = model_config["initializer_scale"],
                    learning_rate = model_config["learning_rate"])

In [ ]:
model.build()

In [ ]:
summary_op = tf.summary.merge(model.summaries)
train_writer = tf.summary.FileWriter(data_config["train_log_dir"],sess.graph)
val_writer = tf.summary.FileWriter(data_config["val_log_dir"],sess.graph)

In [ ]:
#model.initialize_inception_pretrained(sess,INCEPTION_PRETRAINED_CHECKPOINT)

In [ ]:
saver = tf.train.Saver(max_to_keep=20,keep_checkpoint_every_n_hours=1.0)

In [ ]:
model_path = tf.train.latest_checkpoint(data_config["checkpoint_dir"])

In [ ]:
sess = tf.Session()
#sess.run(tf.global_variables_initializer())

In [ ]:
if model_path != None:
    saver.restore(sess,model_path)

In [ ]:
BATCH_SIZE=64
data_gen.init_batch(BATCH_SIZE,"train")

In [ ]:
(data_gen.iter_per_epoch["train"]/2)

In [ ]:
iter_to_save

In [ ]:
infer_util = Inference(model,data_gen.word_to_idx,data_gen.idx_to_word)

In [ ]:
for epoch in range(NUM_EPOCHS):
    for i in range(data_gen.iter_per_epoch["train"]):
        start_time = time.time()
        dataset = data_gen.get_next_batch("train")
        data_gen_time = time.time() - start_time
        
        feed_dict={}
        feed_dict[model.caption_input] = dataset["indexed_caption"]
        feed_dict[model.caption_mask] = dataset["caption_mask"]
        feed_dict[model.rnn_input] = dataset["video"]
        
        if np.mod(i+1, 100) == 0:
            print("Writing Summary")
            summary,loss,global_step,_ = sess.run([summary_op,model.batch_loss,model.global_step,model.train_step],feed_dict=feed_dict)
            train_writer.add_summary(summary, global_step/100)
            
            time_global_step = tf.Summary()
            value = time_global_step.value.add()
            value.simple_value = (time.time() - start_time)
            value.tag = "global_step/time_global_step"
            train_writer.add_summary(time_global_step,global_step/100)
        else:
            loss,global_step,_ = sess.run([model.batch_loss,model.global_step,model.train_step],feed_dict=feed_dict)
        
        print("global_step = ",global_step,
              ", loss = ",loss,
              ', Elapsed time: %.2f' %(time.time() - start_time))        
        
        if np.mod(i+1,data_gen.iter_per_epoch["train"]/2) == 0:
            print("Saving the model ...")
            saver.save(sess, os.path.join(
                data_config["checkpoint_dir"], 'model'), global_step=int(global_step/data_gen.iter_per_epoch[MODE]))

In [ ]:
print("Saving the model ...")
saver.save(sess, os.path.join(
    data_config["checkpoint_dir"], 'model'), global_step=int(global_step/data_gen.iter_per_epoch[MODE]))

In [ ]:
data_gen.init_batch(BATCH_SIZE_TEST,"test")

In [ ]:
dataset = data_gen.get_next_batch("test")

In [ ]:
dataset["gen_caption"] = infer_util.generate_caption_batch(sess,dataset["video"],max_len=30)

In [ ]:
for i in range(10):
    print("File = %s \n  Real caption = %s \n   Gen Caption = %s" %(dataset['file'][i],dataset['caption'][i],dataset['gen_caption'][i]))


## TESTING

In [ ]:
ckpt_path = "/home/ozym4nd145/Coding/Notebook/SURA/code/s2vt_models/train"

In [ ]:
model_names = list(set([n.split(".")[0] for n in os.listdir(ckpt_path) if "model" in n]))

In [ ]:
model_names.sort(key= lambda x: int(x[6:]) )

In [ ]:
model_names

In [ ]:
import re

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split('(\d+)', text) ]


In [ ]:
model_names.sort(key= natural_keys)

In [ ]:
model_names

In [ ]:
natural_keys(model_names[0])

In [ ]:
p = lambda x: int(x[5:])

In [ ]:
p(model_names[0])

In [ ]:
len(data_gen.dataset["test"])

In [ ]:
data_gen.dataset["test"][0]

In [ ]:
data_gen.dataset["test"][0]

In [ ]:
videos = {i["file_name"]:i["path"] for i in data_gen.dataset["train"]}

In [ ]:
len(videos)

In [ ]:
data_gen.init_batch(64,"train")

In [ ]:
data_gen.iter_per_epoch

In [ ]:
2036*5

In [ ]:
for i in videos:
    print(i)

In [ ]:
iter = 0

In [ ]:
range(0,len(videos),50)

In [ ]:
video_paths = {i["file_name"]:i["path"] for i in data_gen.dataset["test"]}
video_files = list(video_paths.keys())
        

In [ ]:
video_files[0]

In [ ]:
len(video_files)

In [ ]:
2990/64

In [ ]:
data_gen.dataset["train"]

## Rough


In [19]:
all_words = data_gen.all_words

In [20]:
import collections

In [21]:
words = collections.Counter(all_words)

In [22]:
x = words.items()

In [23]:
x

dict_items([('dinking', 1), ('complaining', 32), ('textile', 2), ('shrubbery', 1), ('joseph', 2), ('scissors', 39), ('gta5', 2), ('footloose', 15), ('featureless', 2), ('loop', 15), ('rey', 5), ('promises', 3), ('semicircles', 1), ('swimmingh', 1), ('iphones', 2), ('qualification', 1), ('calls', 33), ('soild', 1), ('rapture', 9), ('bell', 34), ('ply', 2), ('grooves', 1), ('cameramen', 3), ('elbows', 1), ('drian', 1), ('hats', 69), ('recap', 21), ('funny', 424), ('wresting', 18), ('jobs', 33), ('putter', 1), ('finds', 61), ('therapeutics', 1), ('harsh', 5), ('resturant', 11), ('trained', 17), ('cardio', 6), ('chance', 13), ('200mbps', 2), ('hypermaniacally', 1), ('heads', 90), ('liquors', 3), ('frictional', 1), ('dr', 116), ('arrival', 4), ('discribing', 5), ('mistakes', 27), ('charger', 6), ('carbreak', 1), ('chair', 647), ('underdog', 3), ('corning', 2), ('grandly', 1), ('akaporva', 1), ('shimmery', 5), ('newsroom', 21), ('unprofessional', 1), ('effectiveness', 2), ('minute', 11), ('n

In [24]:
count_pairs = sorted(x, key=lambda x: (-x[1],x[0]))

In [25]:
count_pairs[-10:]

[('zookeeper', 1),
 ('zoomi', 1),
 ('zoomin', 1),
 ('zoot', 1),
 ('zozo', 1),
 ('zr9000', 1),
 ('zuckerburg', 1),
 ('zuckerburgs', 1),
 ('zukerberb', 1),
 ('zuma', 1)]

In [26]:
def is_ascii(s):
    return all(ord(c) < 128 for c in s)

In [27]:
t = [i for i in count_pairs if not is_ascii(i[0])]

In [28]:
t

[]

In [ ]:
is_ascii("it's")

In [ ]:
import json

In [ ]:
data_config

In [ ]:
with open(data_config["caption_file"],"r") as f:
    xrt = json.loads(f.read())

In [ ]:
t = [(j,i) for j,i in enumerate(xrt["sentences"]) if not is_ascii(i["caption"])]

In [ ]:
t